# Import Data 

In [176]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = 'https://www.basketball-reference.com/leagues/NBA_2023_games.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tbl = soup.find("table",{"id":"schedule"})
schedule = pd.read_html(str(tbl))[0]

months = ["november", "december", "january", "february", "march", "april"]
for  i in months:
    url = 'https://www.basketball-reference.com/leagues/NBA_2023_games-{0}.html'.format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl = soup.find("table",{"id":"schedule"})
    temp_data_frame = pd.read_html(str(tbl))[0]
    schedule = schedule.append(temp_data_frame)

In [177]:
schedule

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,"Tue, Oct 18, 2022",7:30p,Philadelphia 76ers,117.0,Boston Celtics,126.0,Box Score,NaN,19156.0,TD Garden,NaN
1,"Tue, Oct 18, 2022",10:00p,Los Angeles Lakers,109.0,Golden State Warriors,123.0,Box Score,NaN,18064.0,Chase Center,NaN
2,"Wed, Oct 19, 2022",7:00p,Orlando Magic,109.0,Detroit Pistons,113.0,Box Score,NaN,20190.0,Little Caesars Arena,NaN
3,"Wed, Oct 19, 2022",7:00p,Washington Wizards,114.0,Indiana Pacers,107.0,Box Score,NaN,15027.0,Gainbridge Fieldhouse,NaN
4,"Wed, Oct 19, 2022",7:30p,Houston Rockets,107.0,Atlanta Hawks,117.0,Box Score,NaN,17878.0,State Farm Arena,NaN
...,...,...,...,...,...,...,...,...,...,...,...
65,"Sun, Apr 9, 2023",3:30p,Utah Jazz,NaN,Los Angeles Lakers,NaN,NaN,NaN,NaN,Crypto.com Arena,NaN
66,"Sun, Apr 9, 2023",3:30p,New Orleans Pelicans,NaN,Minnesota Timberwolves,NaN,NaN,NaN,NaN,Target Center,NaN
67,"Sun, Apr 9, 2023",3:30p,Memphis Grizzlies,NaN,Oklahoma City Thunder,NaN,NaN,NaN,NaN,Paycom Center,NaN
68,"Sun, Apr 9, 2023",3:30p,Los Angeles Clippers,NaN,Phoenix Suns,NaN,NaN,NaN,NaN,Footprint Center,NaN


In [178]:
schedule.columns

Index(['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1',
       'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes'],
      dtype='object')

# Clean the data

In [179]:
# Make sure all columns have names and the names match coding standards (lowercase with "_" for spaces)
# Get rid of unneeded columns
schedule = schedule.rename(columns = {"PTS":"visitor_pts", "PTS.1":"home_points","Unnamed:7":"OT",'Visitor/Neutral': 'visitor','Home/Neutral': 'home'})
schedule.columns = schedule.columns.str.replace(" ", "_")
schedule.columns = schedule.columns.str.lower()
schedule = schedule.drop(["unnamed:_6", 'notes'], axis = 1)
schedule.columns

Index(['date', 'start_(et)', 'visitor', 'visitor_pts', 'home', 'home_points',
       'unnamed:_7', 'attend.', 'arena'],
      dtype='object')

# Team Report

In [180]:
# parse each team's data 
list_of_team_names = schedule['visitor'].unique()
list_of_team_names

array(['Philadelphia 76ers', 'Los Angeles Lakers', 'Orlando Magic',
       'Washington Wizards', 'Houston Rockets', 'New Orleans Pelicans',
       'New York Knicks', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Oklahoma City Thunder', 'Charlotte Hornets', 'Denver Nuggets',
       'Dallas Mavericks', 'Portland Trail Blazers', 'Milwaukee Bucks',
       'Los Angeles Clippers', 'San Antonio Spurs', 'Toronto Raptors',
       'Boston Celtics', 'Detroit Pistons', 'Memphis Grizzlies',
       'Utah Jazz', 'Phoenix Suns', 'Minnesota Timberwolves',
       'Sacramento Kings', 'Indiana Pacers', 'Brooklyn Nets',
       'Golden State Warriors', 'Atlanta Hawks', 'Miami Heat'],
      dtype=object)

In [181]:
# # Only use nicknames for the teams
# list_of_team_nicknames = []
# for i in list_of_team_names: 
#     array_of_txt = i.split()
#     list_of_team_nicknames.append(array_of_txt[len(array_of_txt)-1])
# list_of_team_nicknames.sort()

In [190]:
list_of_team_dfs = []
for team in list_of_team_names:
    list_of_team_dfs.append(schedule.query('visitor == @team or home == @team'))

In [198]:
df = list_of_team_dfs[1]
df


[                 date start_(et)             visitor  visitor_pts  \
 0   Tue, Oct 18, 2022      7:30p  Philadelphia 76ers        117.0   
 14  Thu, Oct 20, 2022      7:30p     Milwaukee Bucks         90.0   
 27  Sat, Oct 22, 2022      6:00p   San Antonio Spurs        114.0   
 43  Mon, Oct 24, 2022      7:00p      Indiana Pacers        106.0   
 59  Wed, Oct 26, 2022      7:30p  Philadelphia 76ers        109.0   
 ..                ...        ...                 ...          ...   
 12   Sun, Apr 2, 2023      8:00p  Philadelphia 76ers          NaN   
 23   Tue, Apr 4, 2023      8:00p      Boston Celtics          NaN   
 37   Thu, Apr 6, 2023      7:30p          Miami Heat          NaN   
 46   Fri, Apr 7, 2023      7:30p  Philadelphia 76ers          NaN   
 56   Sun, Apr 9, 2023      1:00p  Philadelphia 76ers          NaN   
 
                   home  home_points unnamed:_7  attend.               arena  
 0       Boston Celtics        126.0        NaN  19156.0           TD Garden  


In [199]:
# Calculate basic metrics 

# Calculate winning % 



In [200]:
# Calcualte how they do vs each team 

In [201]:
# Calculate basic metrics for the past 10, 21, 30 days

In [202]:
# calculate prediction for next 5 games 

# Day Report 

In [203]:
# Get all games for the day

In [204]:
# for each game 
# calcualte basic metrics from team report 
# get the prediction for this game 
# get the previous results b/t these 2 teams 
# get how hard their schedule was for the past 10, 21, 30 days